<a href="https://colab.research.google.com/github/decisionenabler-sk/reddit_data_project/blob/main/Reddit_data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install library to scrape Reddit data

In [ ]:
!pip install praw



Import the library and attempt to autheticate user creds

In [9]:
import praw
import datetime
import pytz
# Authenticate your script using your Reddit app credentials

reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    redirect_uri=REDIRECT_URI,
    user_agent=USER_AGENT
)

# You can use the authenticated `reddit` object to interact with Reddit API


In [ ]:
!pip install google-cloud-bigquery
!pip install pandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Setup the dataset and table in bigquery. Define the schema for our table

In [23]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from google.colab import auth


# Set up BigQuery client

auth.authenticate_user()
!gcloud config set project {PROJECT_ID}
client = bigquery.Client(project=PROJECT_ID)
# Set up dataset and table info
dataset_id = 'reddit_otf_data'
table_id = 'intelposts'

# Create dataset (if it doesn't exist)
dataset_ref = client.dataset(dataset_id)
try:
    client.get_dataset(dataset_ref)
except NotFound:
    dataset = bigquery.Dataset(dataset_ref)
    client.create_dataset(dataset)

# Create table (if it doesn't exist)
schema = [
    bigquery.SchemaField('title', 'STRING'),
    bigquery.SchemaField('score', 'INTEGER'),
    bigquery.SchemaField('tag', 'STRING'),
    bigquery.SchemaField('url', 'STRING'),
    bigquery.SchemaField('body', 'STRING'),
    bigquery.SchemaField('createdDate', 'TIMESTAMP'),
    bigquery.SchemaField('upvoteRatio', 'FLOAT'),
    bigquery.SchemaField('numComments', 'INTEGER'),
    bigquery.SchemaField('topComment', 'STRING')
]
table_ref = dataset_ref.table(table_id)
try:
    client.get_table(table_ref)
except NotFound:
    table = bigquery.Table(table_ref, schema=schema)
    client.create_table(table)


Updated property [core/project].


1. Let's look at the subreddit for Orange Theory Workout to get posts related to the community
2. Then we'll store the data to a dataframe, transform the data types of the columns

Now we will store this in a bigquery table

In [ ]:
import pandas as pd
import datetime
# Specify the subreddit you want to extract data from
subreddit_name = 'orangetheory'

# Get the subreddit object
subreddit = reddit.subreddit(subreddit_name)
# Prepare the DataFrame with the data to be inserted
data = []
# Specify the subreddit tag you want to search
tags = ['Early Intel', 'Daily Workout', 'Lift 45 and Tornado Templates']

# Iterate over the posts and insert data into the BigQuery table
for tag in tags:
    intel_posts = subreddit.search(f'flair:"{tag}"', sort='new', limit=10)

    # Iterate over the posts
    for post in intel_posts:
        # Convert the UTC timestamp to PST
        pst_tz = pytz.timezone('US/Pacific')
        utc_dt = datetime.datetime.utcfromtimestamp(post.created_utc)
        pst_dt = utc_dt.replace(tzinfo=pytz.utc).astimezone(pst_tz)

        data.append({
            'title': str(post.title),
            'score': int(post.score),
            'tag': str(post.link_flair_text),
            'url': str(post.url),
            'body': str(post.selftext),
            'createdDate': datetime.datetime.strptime(pst_dt.strftime("%Y-%m-%d %H:%M:%S"), "%Y-%m-%d %H:%M:%S"),
            'upvoteRatio': float(post.upvote_ratio),
            'numComments': int(post.num_comments),
            'topComment': str(post.comments[0].body if len(post.comments) > 0 else "No comments found.")
        })

# Create a DataFrame from the collected data
df = pd.DataFrame(data)



In [35]:
# Get the data types of the DataFrame columns
df.head()
print(df.dtypes)

title                  object
score                   int64
tag                    object
url                    object
body                   object
createdDate    datetime64[ns]
upvoteRatio           float64
numComments             int64
topComment             object
dtype: object


Once the dataframe is ready we'll perform the insert operation in to the big query table

In [36]:
# Load the DataFrame into BigQuery table
job_config = bigquery.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()  # Wait for the job to complete

print(f"Data loaded successfully into BigQuery table: {table_id}")

# Clear the DataFrame
df.drop(df.index, inplace=True)

print("DataFrame cleared.")

Data loaded successfully into BigQuery table: intelposts
DataFrame cleared.
